<a href="https://colab.research.google.com/github/ddeysel27/Crop-Disease-Detection/blob/main/02_train_classifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 02 — Train PlantVillage Disease Classifier


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Set the project directory

In [2]:
PROJECT_DIR = "/content/drive/MyDrive/crop-disease-detection"
ZIP_PATH = f"{PROJECT_DIR}/processed.zip"

In [3]:
import os

if not os.path.exists("/content/processed/PlantDoc"):
    !unzip "$ZIP_PATH" -d /content/


Streaming output truncated to the last 5000 lines.
  inflating: /content/processed/plantVillage/val/Strawberry___Leaf_scorch/1f2420de-a55f-4538-934a-ed289da7a953___RS_L.Scorch 1145.JPG  
  inflating: /content/processed/plantVillage/val/Strawberry___Leaf_scorch/2104ca91-9e5f-4b5c-8981-e02d01fb9268___RS_L.Scorch 9911.JPG  
  inflating: /content/processed/plantVillage/val/Strawberry___Leaf_scorch/220e00d7-57f3-4aa5-b481-9652d44e1642___RS_L.Scorch 1184.JPG  
  inflating: /content/processed/plantVillage/val/Strawberry___Leaf_scorch/24b4756b-eea7-43b9-ace7-8f74e68b8c4f___RS_L.Scorch 0991.JPG  
  inflating: /content/processed/plantVillage/val/Strawberry___Leaf_scorch/253d61ac-85fd-4749-b071-3327f6d9384f___RS_L.Scorch 0082.JPG  
  inflating: /content/processed/plantVillage/val/Strawberry___Leaf_scorch/2828fa61-2c57-48b8-afbc-f28567db7524___RS_L.Scorch 1203.JPG  
  inflating: /content/processed/plantVillage/val/Strawberry___Leaf_scorch/29c3d239-be01-4d55-81fa-37870bc305e9___RS_L.Scorch 0806.JPG

# Load training and validation sets

In [ ]:
# set the data directory
DATA_DIR = "/content/processed/plantVillage"

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms, models, datasets
from torch.optim import Adam
from tqdm import tqdm
import matplotlib.pyplot as plt
import os


In [ ]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader



IMG_SIZE = 224
BATCH_SIZE = 32

train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
])

val_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])

# correctly load train/val folders inside each dataset
train_ds = datasets.ImageFolder(os.path.join(DATA_DIR, "train"), transform=train_tfms)
val_ds   = datasets.ImageFolder(os.path.join(DATA_DIR, "val"), transform=val_tfms)

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_dl   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

# NUM_CLASSES depends on dataset (auto-detect)
NUM_CLASSES = len(train_ds.classes)

train_ds.classes


['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Blueberry___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Cherry_(including_sour)___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Raspberry___healthy',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Strawberry___Leaf_scorch',
 'Strawberry___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_

### Train

In [ ]:
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

LR = 1e-4   # learning rate
EPOCHS = 10

model = models.vit_b_16(weights="IMAGENET1K_V1")
model.heads.head = nn.Linear(model.heads.head.in_features, NUM_CLASSES)
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)


for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for imgs, labels in tqdm(train_dl):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        preds = model(imgs)
        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {total_loss/len(train_dl):.4f}")


Using device: cuda


100%|██████████| 1188/1188 [05:52<00:00,  3.37it/s]


Epoch 1/10 | Train Loss: 0.1989


100%|██████████| 1188/1188 [05:52<00:00,  3.37it/s]


Epoch 2/10 | Train Loss: 0.0677


...

Epoch 8/10 | Train Loss: 0.0324


100%|██████████| 1188/1188 [05:52<00:00,  3.37it/s]


Epoch 9/10 | Train Loss: 0.0305


100%|██████████| 1188/1188 [05:51<00:00,  3.38it/s]

Epoch 10/10 | Train Loss: 0.0257


### Evaluation

In [ ]:
# ---------------------
# Evaluation
# ---------------------
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_dl:
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)

        preds = model(imgs)
        preds = preds.argmax(1)

        correct += (preds == labels).sum().item()
        total += labels.size(0)

val_accuracy = correct / total
print("Validation Accuracy:", val_accuracy * 100, "%")


# ---------------------
# Save Model
# ---------------------
MODEL_NAME = "plant_village_best.pth"   # change per dataset
torch.save(model.state_dict(), MODEL_NAME)

print("Saved:", MODEL_NAME)


In [ ]:
import shutil
SAVE_DIR = "/content/drive/MyDrive/crop-disease-detection/models"
os.makedirs(SAVE_DIR, exist_ok=True)

shutil.copy(MODEL_NAME, f"{SAVE_DIR}/{MODEL_NAME}")
print("Copied to Drive:", f"{SAVE_DIR}/{MODEL_NAME}")


Copied to Drive: /content/drive/MyDrive/crop-disease-detection/models/plant_village_best.pth


# Cassave dataset

In [ ]:
# set the data directory
DATA_DIR = "/content/processed/Cassava"
MODEL_NAME = "cassava_best.pth"


In [ ]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

IMG_SIZE = 224
BATCH_SIZE = 32

train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
])

val_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])

# correctly load train/val folders inside each dataset
train_ds = datasets.ImageFolder(os.path.join(DATA_DIR, "train"), transform=train_tfms)
val_ds   = datasets.ImageFolder(os.path.join(DATA_DIR, "val"), transform=val_tfms)

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_dl   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

# NUM_CLASSES depends on dataset (auto-detect)
NUM_CLASSES = len(train_ds.classes)

train_ds.classes


In [ ]:
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

LR = 1e-4   # learning rate
EPOCHS = 10

model = models.vit_b_16(weights="IMAGENET1K_V1")
model.heads.head = nn.Linear(model.heads.head.in_features, NUM_CLASSES)
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)


for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for imgs, labels in tqdm(train_dl):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        preds = model(imgs)
        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {total_loss/len(train_dl):.4f}")


In [ ]:
# ---------------------
# Evaluation
# ---------------------
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_dl:
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)

        preds = model(imgs)
        preds = preds.argmax(1)

        correct += (preds == labels).sum().item()
        total += labels.size(0)

val_accuracy = correct / total
print("Validation Accuracy:", val_accuracy * 100, "%")


# ---------------------
# Save Model
# ---------------------
MODEL_NAME = "cassava_best.pth"   # change per dataset
torch.save(model.state_dict(), MODEL_NAME)

print("Saved:", MODEL_NAME)


In [ ]:
import shutil
SAVE_DIR = "/content/drive/MyDrive/crop-disease-detection/models"
os.makedirs(SAVE_DIR, exist_ok=True)

shutil.copy(MODEL_NAME, f"{SAVE_DIR}/{MODEL_NAME}")
print("Copied to Drive:", f"{SAVE_DIR}/{MODEL_NAME}")


# Riceleaf

In [ ]:
DATA_DIR = "/content/processed/Rice"
MODEL_NAME = "rice_best.pth"


In [ ]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

IMG_SIZE = 224
BATCH_SIZE = 32

train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
])

val_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])

# correctly load train/val folders inside each dataset
train_ds = datasets.ImageFolder(os.path.join(DATA_DIR, "train"), transform=train_tfms)
val_ds   = datasets.ImageFolder(os.path.join(DATA_DIR, "val"), transform=val_tfms)

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_dl   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

# NUM_CLASSES depends on dataset (auto-detect)
NUM_CLASSES = len(train_ds.classes)

train_ds.classes


In [ ]:
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

LR = 1e-4   # learning rate
EPOCHS = 10

model = models.vit_b_16(weights="IMAGENET1K_V1")
model.heads.head = nn.Linear(model.heads.head.in_features, NUM_CLASSES)
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)


for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for imgs, labels in tqdm(train_dl):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        preds = model(imgs)
        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {total_loss/len(train_dl):.4f}")


In [ ]:
# ---------------------
# Evaluation
# ---------------------
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_dl:
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)

        preds = model(imgs)
        preds = preds.argmax(1)

        correct += (preds == labels).sum().item()
        total += labels.size(0)

val_accuracy = correct / total
print("Validation Accuracy:", val_accuracy * 100, "%")


# ---------------------
# Save Model
# ---------------------
MODEL_NAME = "rice_leaf_best.pth"   # change per dataset
torch.save(model.state_dict(), MODEL_NAME)

print("Saved:", MODEL_NAME)


In [ ]:
import shutil
SAVE_DIR = "/content/drive/MyDrive/crop-disease-detection/models"
os.makedirs(SAVE_DIR, exist_ok=True)

shutil.copy(MODEL_NAME, f"{SAVE_DIR}/{MODEL_NAME}")
print("Copied to Drive:", f"{SAVE_DIR}/{MODEL_NAME}")


PlantDoc for YOLO

In [ ]:
PROJECT_DIR = "/content/drive/MyDrive/crop-disease-detection"
ZIP_PATH = f"{PROJECT_DIR}/PlantDoc.zip"

In [ ]:
import os

if not os.path.exists("/content/drive/MyDrive/crop-disease-detection/PlantDOc"):
    !unzip "$ZIP_PATH" -d /content/


Archive:  /content/drive/MyDrive/crop-disease-detection/PlantDoc.zip
   creating: /content/PlantDoc/
  inflating: /content/PlantDoc/convert_plantdoc_to_yolo.py  
  inflating: /content/PlantDoc/inspect.ipynb  
  inflating: /content/PlantDoc/labels.csv  
  inflating: /content/PlantDoc/plantdoc.yaml  
   creating: /content/PlantDoc/test/
   creating: /content/PlantDoc/test/images/
  inflating: /content/PlantDoc/test/images/%2320+Bacterial+Spot+and+Speck.jpg  
  inflating: /content/PlantDoc/test/images/0.jpg  
  inflating: /content/PlantDoc/test/images/00.jpg  
  inflating: /content/PlantDoc/test/images/000.jpg  
  inflating: /content/PlantDoc/test/images/0000.jpg  
  inflating: /content/PlantDoc/test/images/00pe.jpg  
  inflating: /content/PlantDoc/test/images/01.jpg  
  inflating: /content/PlantDoc/test/images/02.-Rust-2017-207u24s.jpg  
  inflating: /content/PlantDoc/test/images/02c.jpg  
  inflating: /content/PlantDoc/test/images/039b47d574bc4bb8a14259a1cd96a741.jpg  
  inflating: /con

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8s.pt")   # or yolov8s-seg.pt if segmentation

model.train(
    data="/content/PlantDoc/plantdoc.yaml",
    epochs=150,
    imgsz=640,
    batch=16,
    lr0=0.0005,     # lower LR improves stability
    optimizer="AdamW",
    device=0,
)


Ultralytics 8.3.229 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/PlantDoc/plantdoc.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=100, perspective=0.0, p

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c1e8b9c0140>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
import os

for root, dirs, files in os.walk("runs/detect", topdown=True):
    print(root)
    for f in files:
        if f.endswith(".pt"):
            print("   →", f)


runs/detect
runs/detect/plantdoc_detect4
runs/detect/plantdoc_detect4/weights
runs/detect/plantdoc_detect
runs/detect/plantdoc_detect/weights
runs/detect/plantdoc_detect2
runs/detect/plantdoc_detect2/weights
runs/detect/plantdoc_detect3
runs/detect/plantdoc_detect3/weights
runs/detect/plantdoc_detect5
runs/detect/plantdoc_detect5/weights
   → last.pt
   → best.pt
runs/detect/train
runs/detect/train/weights
   → last.pt
   → best.pt


In [ ]:
# export the best model

import shutil

shutil.copy(
    "runs/detect/train/weights/best.pt",
    "/content/drive/MyDrive/crop-disease-detection/models/yolo_plantdoc_detect.pt"
)


'/content/drive/MyDrive/crop-disease-detection/models/yolo_plantdoc_detect.pt'

# Species Classifier on PlantVillage, Cassave, Rice

### Automatic Species Dataset Builder

In [ ]:
import os, shutil

BASE = "/content/processed"
OUTPUT = f"{BASE}/species"
os.makedirs(OUTPUT, exist_ok=True)

def ensure_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


# ------------------------------------------------------------
# 1) PLANTVILLAGE → extract species from folder names
# ------------------------------------------------------------
def collect_plantvillage(path):
    print("Processing PlantVillage...")

    for split in ["train", "val"]:
        split_dir = f"{path}/{split}"
        if not os.path.exists(split_dir):
            continue

        for class_folder in os.listdir(split_dir):
            full = os.path.join(split_dir, class_folder)
            if not os.path.isdir(full):
                continue

            # Extract species name (before the three underscores)
            species = class_folder.split("___")[0]
            species_dir = os.path.join(OUTPUT, species)
            ensure_dir(species_dir)

            # Copy images
            for img in os.listdir(full):
                src = os.path.join(full, img)
                dst = os.path.join(species_dir, img)
                shutil.copy(src, dst)


# ------------------------------------------------------------
# 2) SINGLE-SPECIES DATASETS (Cassava, Rice)
# ------------------------------------------------------------
def collect_single_species_with_diseases(path, species):
    print(f"Processing {species}...")

    species_dir = os.path.join(OUTPUT, species)
    ensure_dir(species_dir)

    for split in ["train", "val"]:
        split_dir = os.path.join(path, split)
        if not os.path.exists(split_dir):
            continue

        # Each subfolder is a disease class, but species is always the same
        for disease_folder in os.listdir(split_dir):
            full = os.path.join(split_dir, disease_folder)
            if not os.path.isdir(full):
                continue

            for img in os.listdir(full):
                src = os.path.join(full, img)
                dst = os.path.join(species_dir, img)
                shutil.copy(src, dst)


# ------------------------------------------------------------
# 3) RUN EVERYTHING
# ------------------------------------------------------------

collect_plantvillage(f"{BASE}/plantVillage")
collect_single_species_with_diseases(f"{BASE}/cassava", "Cassava")
collect_single_species_with_diseases(f"{BASE}/riceleaf", "Rice")

print("Unified species dataset created at:", OUTPUT)


Processing PlantVillage...
Processing Cassava...
Processing Rice...
Unified species dataset created at: /content/processed/species


#

In [ ]:
import os, shutil, random

SOURCE = "/content/processed/species"
OUTPUT = "/content/processed/species_split"

train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

os.makedirs(OUTPUT, exist_ok=True)

for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(OUTPUT, split), exist_ok=True)


def ensure_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


print("Splitting species into train/val/test...")

for species in os.listdir(SOURCE):
    sp_dir = os.path.join(SOURCE, species)
    if not os.path.isdir(sp_dir):
        continue

    images = os.listdir(sp_dir)
    random.shuffle(images)

    n = len(images)
    n_train = int(train_ratio * n)
    n_val = int(val_ratio * n)

    train_imgs = images[:n_train]
    val_imgs   = images[n_train:n_train+n_val]
    test_imgs  = images[n_train+n_val:]

    # create species folders
    for split in ["train", "val", "test"]:
        ensure_dir(os.path.join(OUTPUT, split, species))

    # copy images
    for img in train_imgs:
        shutil.copy(os.path.join(sp_dir, img),
                    os.path.join(OUTPUT, "train", species, img))

    for img in val_imgs:
        shutil.copy(os.path.join(sp_dir, img),
                    os.path.join(OUTPUT, "val", species, img))

    for img in test_imgs:
        shutil.copy(os.path.join(sp_dir, img),
                    os.path.join(OUTPUT, "test", species, img))

print("Done! Split dataset created at:", OUTPUT)


Splitting species into train/val/test...
Done! Split dataset created at: /content/processed/species_split


### Species Classifier (VIT) - Training Code

In [ ]:
# ---------------------------------------------------
# Train ViT Species Classifier (Correct Final Version)
# ---------------------------------------------------

import os
import torch
import torch.nn as nn
from torch.optim import Adam
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm

DATA_DIR = "/content/processed/species_split"
MODEL_NAME = "species_classifier_vit.pth"

IMG_SIZE = 224
BATCH_SIZE = 32
LR = 1e-4
EPOCHS = 12

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)


# ------------------------------
# Transforms
# ------------------------------
train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
])

val_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])

test_tfms = val_tfms


# ------------------------------
# Datasets
# ------------------------------
train_ds = datasets.ImageFolder(os.path.join(DATA_DIR, "train"), transform=train_tfms)
val_ds   = datasets.ImageFolder(os.path.join(DATA_DIR, "val"),   transform=val_tfms)
test_ds  = datasets.ImageFolder(os.path.join(DATA_DIR, "test"),  transform=test_tfms)

NUM_CLASSES = len(train_ds.classes)
print("Detected species:", train_ds.classes)


# ------------------------------
# DataLoaders
# ------------------------------
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_dl   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
test_dl  = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)


# ------------------------------
# ViT Model
# ------------------------------
model = models.vit_b_16(weights="IMAGENET1K_V1")
model.heads.head = nn.Linear(model.heads.head.in_features, NUM_CLASSES)
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)


# ------------------------------
# Training Loop
# ------------------------------
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for imgs, labels in tqdm(train_dl, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        preds = model(imgs)
        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # validation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in val_dl:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            preds = model(imgs).argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total * 100
    print(f"Epoch {epoch+1}: Train Loss={total_loss/len(train_dl):.4f} | Val Acc={val_acc:.2f}%")


# ------------------------------
# Final Test Accuracy
# ------------------------------
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for imgs, labels in test_dl:
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        preds = model(imgs).argmax(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print("Test Accuracy:", correct / total * 100, "%")


# ------------------------------
# Save model
# ------------------------------
torch.save(model.state_dict(), MODEL_NAME)
print("Saved:", MODEL_NAME)


Using device: cuda
Detected species: ['Apple', 'Blueberry', 'Cassava', 'Cherry_(including_sour)', 'Corn_(maize)', 'Grape', 'Orange', 'Peach', 'Pepper,_bell', 'Potato', 'Raspberry', 'Rice', 'Soybean', 'Squash', 'Strawberry', 'Tomato']
Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth


100%|██████████| 330M/330M [00:01<00:00, 243MB/s]
Epoch 1/12: 100%|██████████| 1683/1683 [10:21<00:00,  2.71it/s]


Epoch 1: Train Loss=0.0729 | Val Acc=97.62%


Epoch 2/12: 100%|██████████| 1683/1683 [10:22<00:00,  2.71it/s]


Epoch 2: Train Loss=0.0279 | Val Acc=98.62%


...

Epoch 3: Train Loss=0.0263 | Val Acc=99.51%


Epoch 4/12: 100%|██████████| 1683/1683 [10:23<00:00,  2.70it/s]


Epoch 4: Train Loss=0.0215 | Val Acc=99.02%


Epoch 5/12:   9%|▉         | 149/1683 [00:55<09:28,  2.70it/s]


KeyboardInterrupt: 

In [ ]:
!zip -r species_split.zip /content/processed/species_split


Streaming output truncated to the last 5000 lines.
  adding: content/processed/species_split/test/Tomato/68d28b97-72a6-41e1-8195-4afa7fa7169d___Com.G_TgS_FL 7986.JPG (deflated 1%)
  adding: content/processed/species_split/test/Tomato/deafaefa-dc4f-40ab-b26a-f462d334e8b2___GCREC_Bact.Sp 3300.JPG (deflated 2%)
  adding: content/processed/species_split/test/Tomato/80817b48-d9dc-4b29-b6ac-1f7bb58e3d00___RS_Late.B 4833.JPG (deflated 1%)
  adding: content/processed/species_split/test/Tomato/1ea54b98-1f87-4da7-9bc2-f6460f18b376___RS_Late.B 6713.JPG (deflated 1%)
  adding: content/processed/species_split/test/Tomato/55339b80-574d-4379-bc85-35558fcc0bea___UF.GRC_BS_Lab Leaf 0554.JPG (deflated 1%)
  adding: content/processed/species_split/test/Tomato/0f514b75-2c74-4075-b9c7-5d469ab292ba___Com.G_SpM_FL 1205.JPG (deflated 1%)
  adding: content/processed/species_split/test/Tomato/5e70b8f4-26db-4ac4-a23a-dac4fd0be93a___YLCV_GCREC 5401.JPG (deflated 2%)
  adding: content/processed/species_split/test/

In [10]:
ROOT = "/content/drive/MyDrive/crop-disease-detection"

MODELS_DIR = f"{ROOT}/models"
DATA_DIR   = f"{ROOT}/data/processed"


In [23]:
import torch
import timm
from pathlib import Path

DEVICE = "cpu"

from torchvision import models
import torch.nn as nn

def _build_vit_base(num_classes: int):
    model = models.vit_b_16(weights=None)   # architecture only
    in_features = model.heads.head.in_features
    model.heads.head = nn.Linear(in_features, num_classes)
    return model


def load_vit_model(weights_path: str, num_classes: int):
    model = _build_vit_base(num_classes)
    state_dict = torch.load(weights_path, map_location=DEVICE)
    model.load_state_dict(state_dict)
    model.eval()
    return model

def load_disease_model(weights_path: str, num_classes: int):
    return load_vit_model(weights_path, num_classes=num_classes)


In [24]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, classification_report

def evaluate_imagefolder_model(model_path, test_dir):
    # Detect number of disease classes from folder names
    class_names = sorted(os.listdir(test_dir))
    num_classes = len(class_names)

    print("Detected disease classes:", class_names)

    # Load model
    model = load_disease_model(model_path, num_classes=num_classes)

    # Transforms
    tfms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

    # Dataset
    ds = datasets.ImageFolder(test_dir, transform=tfms)
    dl = DataLoader(ds, batch_size=1, shuffle=False)

    y_true = []
    y_pred = []

    for img, label in dl:
        with torch.no_grad():
            pred = model(img)
            pred_idx = pred.argmax(dim=1).item()

        y_true.append(label.item())
        y_pred.append(pred_idx)

    # Metrics
    print("\nCONFUSION MATRIX:")
    print(confusion_matrix(y_true, y_pred))

    print("\nCLASSIFICATION REPORT:")
    print(classification_report(y_true, y_pred, target_names=class_names))


In [37]:
cassava_labels = [
    "Cassava Bacterial Blight (CBB)",
    "Cassava Brown Streak Disease (CBSD)",
    "Cassava Green Mottle (CGM)",
    "Cassava Mosaic Disease (CMD)",
    "Healthy"
]


In [35]:
model = _build_vit_base(num_classes=5)
model.load_state_dict(torch.load("/content/drive/MyDrive/crop-disease-detection/models/cassava_best.pth", map_location="cpu"))


<All keys matched successfully>

In [38]:
cassava_model = f"/content/drive/MyDrive/crop-disease-detection/models/cassava_best.pth"
cassava_test = "/content/processed/Cassava/test"


evaluate_imagefolder_model(cassava_model, cassava_test)


Detected disease classes: ['Cassava Bacterial Blight (CBB)', 'Cassava Brown Streak Disease (CBSD)', 'Cassava Green Mottle (CGM)', 'Cassava Mosaic Disease (CMD)', 'Healthy']

CONFUSION MATRIX:
[[  48   24    8    7   22]
 [  20  154   13   17   15]
 [   5   12  159   46   16]
 [   7   31   28 1232   18]
 [  34   32   34   39  119]]

CLASSIFICATION REPORT:
                                     precision    recall  f1-score   support

     Cassava Bacterial Blight (CBB)       0.42      0.44      0.43       109
Cassava Brown Streak Disease (CBSD)       0.61      0.70      0.65       219
         Cassava Green Mottle (CGM)       0.66      0.67      0.66       238
       Cassava Mosaic Disease (CMD)       0.92      0.94      0.93      1316
                            Healthy       0.63      0.46      0.53       258

                           accuracy                           0.80      2140
                          macro avg       0.65      0.64      0.64      2140
                       we

## PlantVillage

In [44]:
pv_labels = [
    "Cassava Bacterial Blight (CBB)",
    "Cassava Brown Streak Disease (CBSD)",
    "Cassava Green Mottle (CGM)",
    "Cassava Mosaic Disease (CMD)",
    "Healthy"
]

model = _build_vit_base(num_classes=38)
model.load_state_dict(torch.load("/content/drive/MyDrive/crop-disease-detection/models/plant_village_best.pth", map_location="cpu"))

cassava_model = f"/content/drive/MyDrive/crop-disease-detection/models/plant_village_best.pth"
cassava_test = "/content/processed/plantVillage/test"


evaluate_imagefolder_model(cassava_model, cassava_test)




Detected disease classes: ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_

### Rice

In [43]:
rice_labels = [
    "leaf_blast",
    "brown_spot",
    "bacterial_leaf_blight",
    "leaf_scald",
    "narrow_brown_spot",
    "healthy"
]

model = _build_vit_base(num_classes=6)
model.load_state_dict(torch.load("/content/drive/MyDrive/crop-disease-detection/models/rice_leaf_best.pth", map_location="cpu"))

cassava_model = f"/content/drive/MyDrive/crop-disease-detection/models/rice_leaf_best.pth"
cassava_test = "/content/processed/riceleaf/test"


evaluate_imagefolder_model(cassava_model, cassava_test)




Detected disease classes: ['bacterial_leaf_blight', 'brown_spot', 'healthy', 'leaf_blast', 'leaf_scald', 'narrow_brown_spot']

CONFUSION MATRIX:
[[34  1  0  0  0  0]
 [ 0 32  0  2  0  1]
 [ 0  0 35  0  0  0]
 [ 0  7  2 26  0  0]
 [ 0  0  0  0 35  0]
 [ 0  0  0  0  2 33]]

CLASSIFICATION REPORT:
                       precision    recall  f1-score   support

bacterial_leaf_blight       1.00      0.97      0.99        35
           brown_spot       0.80      0.91      0.85        35
              healthy       0.95      1.00      0.97        35
           leaf_blast       0.93      0.74      0.83        35
           leaf_scald       0.95      1.00      0.97        35
    narrow_brown_spot       0.97      0.94      0.96        35

             accuracy                           0.93       210
            macro avg       0.93      0.93      0.93       210
         weighted avg       0.93      0.93      0.93       210

